In [14]:
import pandas as pd
import os
import numpy as np
import re

from sklearn.preprocessing import scale
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from scipy.stats import expon
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, KFold

from gensim.utils import simple_preprocess
from sklearn.metrics import classification_report
#import glob
#model = api.load('word2vec-google-news-300')

In [2]:
df = pd.read_csv('train.csv')
df_test_gold = pd.read_csv('test_gold.csv')

In [3]:
df

,index,tweet,humor,prejudice_woman,prejudice_lgbtiq,prejudice_inmigrant_race,gordofobia,mean_prejudice
0,72157,Mi celular tiene una aplicación que te hace ve...,1,0,0,0,1,3.0
1,68084,"En esta vida me tocó tener mala suerte, espero...",1,0,0,0,1,2.8
2,69089,"Tu mamá es taaan taan obesa, que cuando pasa f...",1,0,0,0,1,3.6
3,69190,Mi tía me dijo: \n- tengo memoria de Elefante....,1,0,0,0,1,3.4
4,70474,"- Mamá, en el colegio me dicen gorda.\n- ¡Ay M...",1,0,0,0,1,3.0
...,...,...,...,...,...,...,...,...
2666,41280,Un claro ejemplo más del vacío moral de las fe...,0,1,0,0,0,3.4
2667,2166,MENTION Vamos a preguntar a las feminazis. Par...,0,1,0,0,0,3.8
2668,39933,"Si tuviera tetas y subiera fotos picantes, al ...",0,1,0,0,0,4.0
2669,4992,qtagarre dl culo ynotarle toda lapolla ay bien...,0,1,0,0,0,3.8


In [4]:
# Removing the stop words
from gensim.parsing.preprocessing import remove_stopwords

def preprocess( doc):
    doc = re.sub('\W+',' ', doc.lower())
    doc = doc.replace('\n','')
    return doc

In [5]:
df_test_gold['clean'] = [simple_preprocess(preprocess(doc)) for doc in df_test_gold['tweet']]

In [6]:
df['clean'] = [simple_preprocess(preprocess(doc)) for doc in df['tweet']]
df['clean']

0       [mi, celular, tiene, una, aplicación, que, te,...
1       [en, esta, vida, me, tocó, tener, mala, suerte...
2       [tu, mamá, es, taaan, taan, obesa, que, cuando...
3       [mi, tía, me, dijo, tengo, memoria, de, elefan...
4       [mamá, en, el, colegio, me, dicen, gorda, ay, ...
                              ...                        
2666    [un, claro, ejemplo, más, del, vacío, moral, d...
2667    [mention, vamos, preguntar, las, feminazis, pa...
2668    [si, tuviera, tetas, subiera, fotos, picantes,...
2669    [qtagarre, dl, culo, ynotarle, toda, lapolla, ...
2670    [claro, internet, lo, que, merece, son, fotos,...
Name: clean, Length: 2671, dtype: object

# Task 1:

HUrtful HUmour Detection:

The first subtask consists in determining whether a prejudicial tweet is intended to cause humour. Participants will have to distinguish between tweets that using humour express prejudice and tweets that express prejudice without using humour. For this, the systems will be evaluated and ranked employing the F1-measure over the positive class.

Vamos a evaluar el desempeño de distintos embeddings

In [7]:
#use 1 for positive sentiment, 0 for negative
x_train, x_test, y_train, y_test = train_test_split( df['clean'], df['humor'], test_size=0.2, random_state = 0)

# FastText

In [8]:
import fasttext
import fasttext.util
##fasttext.util.download_model('es', if_exists='ignore')  # Spanish
ft = fasttext.load_model('cc.es.300.bin')

In [9]:
n_dim = 300

model = ft

def get_embedding(word):
    try:
        embedding = model[word]
    except:
        embedding = np.zeros((n_dim,))
    return embedding


In [10]:
#X_train_embeddings =[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in x_train]
X_test_embeddings = [np.mean(np.array(list(map(get_embedding, tok_sent))),axis=0) for tok_sent in x_test]
X_train_embeddings =[np.mean(np.array(list(map(get_embedding, tok_sent))), axis=0) for tok_sent in x_train]

X_test_gold_embeddings =[np.mean(np.array(list(map(get_embedding, tok_sent))), axis=0) for tok_sent in df_test_gold.clean]

## Evaluación 

### LogisticRegression

In [ ]:
from scipy.stats import expon
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, KFold

pipeline = make_pipeline(StandardScaler(), 
                         LogisticRegression(random_state=1, 
                                             #class_weight={0:1, 1:2},
                                            class_weight='balanced', 
                                            max_iter = 4000,
                                             solver='saga'))
#
# Create a randomized search for finding most appropriate model
#
param_distributions = [{'logisticregression__C': expon(scale=100), 
                        'logisticregression__penalty': ['l1', 'l2', 'elasticnet', None]}]

rs = RandomizedSearchCV(estimator = pipeline, 
                        param_distributions = param_distributions, 
                        cv = 5, 
                        scoring = 'f1', 
                        refit = True, 
                        n_jobs = -1,
                        random_state=1)

In [ ]:
rs.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
print('Best Score:', rs.best_score_, '\nBest Params:', rs.best_params_)
print('Train F1 logisticregression CV: %0.5f' % rs.score(X_train_embeddings, y_train))
print('Test F1 logisticregression test: %0.5f' % rs.score(X_test_embeddings, y_test))

In [ ]:
# Get predictions for test data
y_test_pred = rs.predict(X_train_embeddings)
print(classification_report(y_train , y_test_pred))

In [ ]:
# Get predictions for test data
y_test_pred = rs.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

In [ ]:
# Get predictions for test data
y_test_pred = rs.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

### SVC

In [11]:
## SVC
from sklearn.svm import SVC


pipeline = make_pipeline(StandardScaler(), 
                         SVC(random_state=1, 
                             class_weight={0:1, 1:2}
                              )
                          )
#
# Create a randomized search for finding most appropriate model
#
param_distributions = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
                        'svc__C': expon(scale=100)}]


rs = RandomizedSearchCV(estimator = pipeline, 
                        param_distributions = param_distributions, 
                        cv = 5, 
                        scoring = 'f1', 
                        refit = True, 
                        n_jobs = 1,
                        random_state=1)

In [12]:
rs.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
print('Best Score:', rs.best_score_, '\nBest Params:', rs.best_params_)
print('Train F1 svc CV: %0.5f' % rs.score(X_train_embeddings, y_train))
print('Test F1 SVC test: %0.5f' % rs.score(X_test_embeddings, y_test))
#print(classification_report(y_test ,y_test_pred))

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Angel\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Angel\anaconda3\lib\site-packages\sklearn\pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Angel\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 217, in fit
    raise ValueError(
ValueError: Precomputed matrix must be a square

Best Score: 0.6589534758879733 
Best Params: {'svc__C': 50.54525417107485, 'svc__kernel': 'rbf'}
Train F1 svc CV: 0.99292
Test F1 SVC test: 0.60927


In [15]:
# Get predictions for test data
y_test_pred = rs.predict(X_train_embeddings)
print('Train SVC')
print(classification_report(y_train , y_test_pred))


print('Test SVC')
# Get predictions for test data
y_test_pred = rs.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

print('Test gold SVC')
# Get predictions for test data
y_test_pred = rs.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

Train SVC
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1433
           1       0.99      1.00      0.99       703

    accuracy                           1.00      2136
   macro avg       0.99      1.00      0.99      2136
weighted avg       1.00      1.00      1.00      2136

Test SVC
              precision    recall  f1-score   support

           0       0.81      0.88      0.85       369
           1       0.68      0.55      0.61       166

    accuracy                           0.78       535
   macro avg       0.75      0.72      0.73       535
weighted avg       0.77      0.78      0.77       535

Test gold SVC
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       522
           1       0.75      0.69      0.72       256

    accuracy                           0.82       778
   macro avg       0.80      0.79      0.79       778
weighted avg       0.82      0.82      0.

### XGBoost

In [17]:
from scipy import stats
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

pipeline = make_pipeline(StandardScaler(), XGBClassifier(objective = 'binary:logistic'))


#clf_xgb = XGBClassifier(objective = 'binary:logistic')
param_dist = {'xgbclassifier__n_estimators': stats.randint(150, 500),
              'xgbclassifier__learning_rate': stats.uniform(0.01, 0.07),
              'xgbclassifier__subsample': stats.uniform(0.3, 0.7),
              'xgbclassifier__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
              'xgbclassifier__colsample_bytree': stats.uniform(0.5, 0.45),
              'xgbclassifier__min_child_weight': [1, 2, 3, 4]
             }

rs = RandomizedSearchCV(estimator = pipeline, 
                        param_distributions = param_dist, 
                        cv = 5, 
                        scoring = 'f1', 
                        refit = True, 
                        random_state = 0, 
                        n_jobs = -1)

In [18]:
rs.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
print('Best Score:', rs.best_score_, '\nBest Params:', rs.best_params_)
print('Train F1 logisticregression CV: %0.5f' % rs.score(X_train_embeddings, y_train))
print('Test F1 logisticregression test: %0.5f' % rs.score(X_test_embeddings, y_test))
#print(classification_report(y_test ,y_test_pred))

Best Score: 0.6544442465063672 
Best Params: {'xgbclassifier__colsample_bytree': 0.5608481780324535, 'xgbclassifier__learning_rate': 0.03268987054552499, 'xgbclassifier__max_depth': 8, 'xgbclassifier__min_child_weight': 3, 'xgbclassifier__n_estimators': 394, 'xgbclassifier__subsample': 0.5705422867881034}
Train F1 logisticregression CV: 0.99288
Test F1 logisticregression test: 0.60714


In [19]:
# Get predictions for test data
y_test_pred = rs.predict(X_train_embeddings)
print('Train xgboost')
print(classification_report(y_train , y_test_pred))


print('Test xgboost')
# Get predictions for test data
y_test_pred = rs.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

print('Test gold xgboost')
# Get predictions for test data
y_test_pred = rs.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

Train xgboost
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1433
           1       0.99      0.99      0.99       703

    accuracy                           1.00      2136
   macro avg       1.00      0.99      0.99      2136
weighted avg       1.00      1.00      1.00      2136

Test xgboost
              precision    recall  f1-score   support

           0       0.81      0.92      0.86       369
           1       0.75      0.51      0.61       166

    accuracy                           0.79       535
   macro avg       0.78      0.72      0.73       535
weighted avg       0.79      0.79      0.78       535

Test gold xgboost
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       522
           1       0.77      0.64      0.70       256

    accuracy                           0.82       778
   macro avg       0.80      0.77      0.79       778
weighted avg       0.82      

### RandomForest

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=0)



param_grid = { 
    'n_estimators': [100, 200, 300, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, 
                      param_grid=param_grid, 
                      cv= 5)

CV_rfc.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
print('Best Score:', CV_rfc.best_score_, '\nBest Params:', CV_rfc.best_params_)
print('Train F1 logisticregression CV: %0.5f' % CV_rfc.score(X_train_embeddings, y_train))
print('Test F1 logisticregression test: %0.5f' % CV_rfc.score(X_test_embeddings, y_test))

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\Angel\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

Best Score: 0.7645177176125545 
Best Params: {'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'n_estimators': 300}
Train F1 logisticregression CV: 0.94054
Test F1 logisticregression test: 0.78131


In [21]:
# Get predictions for test data
y_test_pred = CV_rfc.predict(X_train_embeddings)
print('Train CV_rfc')
print(classification_report(y_train , y_test_pred))


print('Test CV_rfc')
# Get predictions for test data
y_test_pred = CV_rfc.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

print('Test gold CV_rfc')
# Get predictions for test data
y_test_pred = CV_rfc.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

Train CV_rfc
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1433
           1       1.00      0.82      0.90       703

    accuracy                           0.94      2136
   macro avg       0.96      0.91      0.93      2136
weighted avg       0.94      0.94      0.94      2136

Test CV_rfc
              precision    recall  f1-score   support

           0       0.77      0.96      0.86       369
           1       0.83      0.37      0.51       166

    accuracy                           0.78       535
   macro avg       0.80      0.67      0.69       535
weighted avg       0.79      0.78      0.75       535

Test gold CV_rfc
              precision    recall  f1-score   support

           0       0.76      0.94      0.84       522
           1       0.78      0.41      0.53       256

    accuracy                           0.77       778
   macro avg       0.77      0.67      0.69       778
weighted avg       0.77      0.7

### Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB

pipeNB = make_pipeline(StandardScaler(), GaussianNB())

pipeNB.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
#print('Best Score:', pipeNB.best_score_, '\nBest Params:', pipeNB.best_params_)
print('Train F1 logisticregression CV: %0.5f' % pipeNB.score(X_train_embeddings, y_train))
print('Test F1 logisticregression test: %0.5f' % pipeNB.score(X_test_embeddings, y_test))

# Get predictions for test data
print('\nTrain GaussianNB')
y_test_pred = pipeNB.predict(X_train_embeddings)
print(classification_report(y_train , y_test_pred))

print('\nTest GaussianNB')
# Get predictions for test data
y_test_pred = pipeNB.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

print('\nTest gold GaussianNB')
# Get predictions for test data
y_test_pred = pipeNB.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

Train F1 logisticregression CV: 0.70927
Test F1 logisticregression test: 0.70654

Train GaussianNB
              precision    recall  f1-score   support

           0       0.81      0.74      0.77      1433
           1       0.55      0.65      0.60       703

    accuracy                           0.71      2136
   macro avg       0.68      0.69      0.68      2136
weighted avg       0.73      0.71      0.71      2136


Test GaussianNB
              precision    recall  f1-score   support

           0       0.83      0.73      0.77       369
           1       0.52      0.66      0.58       166

    accuracy                           0.71       535
   macro avg       0.67      0.69      0.68       535
weighted avg       0.73      0.71      0.71       535


Test gold GaussianNB
              precision    recall  f1-score   support

           0       0.78      0.77      0.78       522
           1       0.55      0.57      0.56       256

    accuracy                           0.71 

In [23]:
from sklearn.naive_bayes import BernoulliNB

pipeNB = make_pipeline(StandardScaler(), BernoulliNB())

pipeNB.fit(X_train_embeddings, y_train)
# Find the best score, params and f1 on the test dataset
#
#print('Best Score:', pipeNB.best_score_, '\nBest Params:', pipeNB.best_params_)
print('Train F1 logisticregression CV: %0.5f' % pipeNB.score(X_train_embeddings, y_train))
print('Test F1 logisticregression test: %0.5f' % pipeNB.score(X_test_embeddings, y_test))

# Get predictions for test data
print('\nTrain BernoulliNB')
y_test_pred = pipeNB.predict(X_train_embeddings)
print(classification_report(y_train , y_test_pred))

print('\nTest BernoulliNB')
# Get predictions for test data
y_test_pred = pipeNB.predict(X_test_embeddings)
print(classification_report(y_test , y_test_pred))

print('\nTest gold BernoulliNB')
# Get predictions for test data
y_test_pred = pipeNB.predict(X_test_gold_embeddings)
print(classification_report(df_test_gold.humor , y_test_pred))

Train F1 logisticregression CV: 0.67275
Test F1 logisticregression test: 0.66168

Train BernoulliNB
              precision    recall  f1-score   support

           0       0.80      0.68      0.74      1433
           1       0.50      0.66      0.57       703

    accuracy                           0.67      2136
   macro avg       0.65      0.67      0.65      2136
weighted avg       0.70      0.67      0.68      2136


Test BernoulliNB
              precision    recall  f1-score   support

           0       0.81      0.67      0.73       369
           1       0.47      0.64      0.54       166

    accuracy                           0.66       535
   macro avg       0.64      0.66      0.64       535
weighted avg       0.70      0.66      0.67       535


Test gold BernoulliNB
              precision    recall  f1-score   support

           0       0.79      0.74      0.76       522
           1       0.52      0.59      0.55       256

    accuracy                           0.

## word2vec

In [ ]:
# opción 2: cargar una cantidad fija de vectores (más rápido dependiendo de la cantidad cargada)
from gensim.models.keyedvectors import KeyedVectors

wordvectors_file_vec = '/home/posgrado/Documentos/Posgrado/Mineria/embeddings_spanish/SBW-vectors-300-min5.bin/sbw_vectors.bin'
#cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [ ]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVectorKeyed(model, text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:


#Initialize model and build vocab
imdb_w2v = Word2Vec(vector_size = n_dim, min_count = 2)
imdb_w2v.build_vocab(x_train)

#Train the model over train_reviews (this may take several minutes)
imdb_w2v.train(x_train, total_examples = imdb_w2v.corpus_count, epochs = imdb_w2v.epochs)

In [ ]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVector(model, text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
#Build test tweet vectors then scale
test_vecs = np.concatenate([buildWordVector(imdb_w2v,z, n_dim) for z in x_test])
test_vecs = scale(test_vecs)

train_vecs = np.concatenate([buildWordVector(imdb_w2v,z, n_dim) for z in x_train])
train_vecs = scale(train_vecs)

In [ ]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier


lr = SGDClassifier(loss='log_loss', penalty='l1')
lr.fit(train_vecs, y_train)

y_predict = lr.predict(test_vecs)

print ('Test F1_score: %.2f'%f1_score(y_test, y_predict))

In [ ]:
#Create ROC curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

pred_probas = lr.predict_proba(test_vecs)[:,1]
fpr,tpr,_ = roc_curve(y_test, pred_probas)
roc_auc = auc(fpr,tpr)
plt.plot(fpr,tpr,label='area = %.2f' %roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')
plt.show()

# Word2vec pre-entrenado

In [ ]:
# Word2vec pre-entrenado

import gensim.downloader as api
#model = api.load('word2vec-google-news-300')

#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVectorKeyed(model, text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

#train_vecs = np.concatenate([buildWordVectorKeyed(model,z, n_dim) for z in x_train])
#train_vecs_scale = scale(train_vecs)
#test_vecs = np.concatenate([buildWordVectorKeyed(model,z, n_dim) for z in x_test])
#test_vecs_sacale = scale(test_vecs)



In [ ]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
lr = SGDClassifier(loss='log_loss', penalty='l1')
lr.fit(train_vecs_sacale, y_train)

y_predict = lr.predict(test_vecs_sacale)

print ('Test F1_score: %.2f'%f1_score(y_test_scale, y_predict))

In [ ]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log_loss', penalty='l1')
lr.fit(train_vecs_scale, y_train)

y_predict = lr.predict(test_vecs)

print ('Test F1 score SGDClassifier: %.2f'%f1_score(y_test, y_predict))


from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_vecs, y_train)

y_predict = clf.predict(test_vecs)

print ('Test F1 score SVC: %.2f'%f1_score(y_test, y_predict))


from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(train_vecs, y_train)

y_predict = clf.predict(test_vecs)

print ('Test F1 score RandomForestClassifier: %.2f'%f1_score(y_test, y_predict))


import sklearn as sk
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1, max_iter = 1000)
NN.fit(train_vecs_scale, y_train)
y_predict = NN.predict(test_vecs)

print ('Test F1 score MLPClassifier: %.2f'%f1_score(y_test, y_predict))

## Doc2Vec

In [ ]:
import random
size = 400
#instantiate our DM and DBOW models
model_dm   = Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, workers=3)
model_dbow = Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, dm=0, workers=3)

In [ ]:
documents = [ TaggedDocument(doc, [i]) for i, doc in enumerate(df['clean'])]

In [ ]:
model_dm.build_vocab(documents)
model_dbow.build_vocab(documents)

In [ ]:
model_dm.train(documents, total_examples=model_dm.corpus_count, epochs=model_dm.epochs)
model_dbow.train(documents, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)

In [ ]:
#get vectors by model
train_vec_dm  = np.concatenate([ model_dm.infer_vector(doc_tokenized).reshape((1, size)) for doc_tokenized in x_train ])
train_vec_dbow = np.concatenate([model_dbow.infer_vector(doc_tokenized).reshape((1, size)) for doc_tokenized in x_train])

train_vec = np.hstack((train_vec_dm, train_vec_dbow))
train_vec_scale = scale(train_vec)

In [ ]:
#get vectors by model
test_vec_dm  = np.concatenate([ model_dm.infer_vector(doc_tokenized).reshape((1, size)) for doc_tokenized in x_test])
test_vec_dbow = np.concatenate([model_dbow.infer_vector(doc_tokenized).reshape((1, size)) for doc_tokenized in x_test])

test_vec = np.hstack((test_vec_dm, test_vec_dbow))

test_vec_scale = scale(test_vec)

In [ ]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log_loss', penalty='l1')
lr.fit(train_vec_scale, y_train)

y_predict = lr.predict(test_vec)

print ('Test F1 score: %.2f'%f1_score(y_test, y_predict))


In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_vec, y_train)

y_predict = clf.predict(test_vec)

print ('Test F1 score: %.2f'%f1_score(y_test, y_predict))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(train_vec, y_train)

y_predict = clf.predict(test_vec)

print ('Test F1 score: %.2f'%f1_score(y_test, y_predict))


In [ ]:
import sklearn as sk
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1, max_iter = 1000)
NN.fit(train_vec_scale, y_train)
y_predict = NN.predict(test_vec)

print ('Test F1 score: %.2f'%f1_score(y_test, y_predict))

In [ ]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression

clf = make_pipeline(MinMaxScaler(), LogisticRegression(max_iter = 3000))
clf.fit(train_vec, y_train)

y_predict = clf.predict(test_vec)

print ('Test F1 score: %.2f'%f1_score(y_test, y_predict))


# FastText

In [ ]:
#fastText
model = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
n_dim = 300
train_vecs = np.concatenate([buildWordVectorKeyed(model,z, n_dim) for z in x_train])
train_vecs_scale = scale(train_vecs)

test_vecs = np.concatenate([buildWordVectorKeyed(model,z, n_dim) for z in x_test])
test_vecs_sacale = scale(test_vecs)

In [ ]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log_loss', penalty='l1')
lr.fit(train_vecs_scale, y_train)

y_predict = lr.predict(test_vecs)

print ('Test F1 score SGDClassifier: %.2f'%f1_score(y_test, y_predict))


from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_vecs, y_train)

y_predict = clf.predict(test_vecs)

print ('Test F1 score SVC: %.2f'%f1_score(y_test, y_predict))


from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(train_vecs, y_train)

y_predict = clf.predict(test_vecs)

print ('Test F1 score RandomForestClassifier: %.2f'%f1_score(y_test, y_predict))


import sklearn as sk
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1, max_iter = 1000)
NN.fit(train_vecs_scale, y_train)
y_predict = NN.predict(test_vecs)

print ('Test F1 score MLPClassifier: %.2f'%f1_score(y_test, y_predict))

## XGBoost

In [ ]:
pip install --user xgboost

In [ ]:
import xgboost as xgb

In [ ]:
X, y =df['clean'], df['humor']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(np.array(y).reshape(-1, 1))


# Train a model using the scikit-learn API
xgb_classifier = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3)
xgb_classifier.fit(X_train, y_train)

# Convert the model to a native API model
#model = xgb_classifier.get_booster()


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

In [ ]:
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
#Glove
model = api.load('glove-wiki-gigaword-300')

In [ ]:
# install the transformers library
!pip install transformers

# import the necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification



In [ ]:
# load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# define the input text and convert it to input features
text = "This is a positive review."
input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # batch size 1

# make the prediction
output = model(input_ids)
prediction = output[0].argmax().item()

# print the prediction
print(f'Prediction: {prediction}')

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text


In [ ]:
!pip install -q tf-models-official


## Task 2A:

Prejudice Target Detection:

Taking into account the minority groups analyzed, i.e, Women and feminists, LGBTIQ community and Immigrants, racially discriminated people, and overweight people,  participants are asked to identify the targeted groups on each tweet as a multilabel classification task.

The metric employed for the second task will be macro-F1.

## Task 2B:

Degree of Prejudice Prediction:

The third subtask consists of predicting on a continuous scale from 1 to 5 to evaluate how prejudicial the message is on average among minority groups. We will evaluate the submitted predictions employing the Root Mean Squared Error. 